##Importing Necessary libraries

In [3]:
import pandas as pd
import numpy as np

Importing the Excel file

In [40]:
from google.colab import files

files.upload()

Saving 2024_PersonalityTraits_SurveyData.xls to 2024_PersonalityTraits_SurveyData.xls


{'2024_PersonalityTraits_SurveyData.xls': b'PK\x03\x04\x14\x00\x00\x00\x08\x00j4\x7fT\x07AMb\x81\x00\x00\x00\xb1\x00\x00\x00\x10\x00\x00\x00docProps/app.xmlM\x8e=\x0b\x021\x10D\xff\xcaq\xbd\xb7A\xc1Bb@\xd0R\xb0\xb2\x0f{\x1b/\x90dC\xb2B~\xbe9\xc1\x8fn\x1eo\x18F\xdf\ng*\xe2\xa9\x0e-\x86T\x8f\xe3"\x92\x0f\x00\x15\x17\x8a\xb6N]\xa7n\x1c\x97h\xa5cy\x00;\xe7\x91\xce\x8c\xcfHI`\xab\xd4\x1e\xa8\t\xa5\x99\xe6M\xfe\x0e\x8eF\x9fr\x0e\x1e\xadxN\xe6\xea\xb1pe\'\xc3\xa5!\x05\r\xffrm\xde\xa9\xd45\xef&\xf5\x96\x1f\xd6\xf0;i^PK\x03\x04\x14\x00\x00\x00\x08\x00j4\x7fTU\xf1\xb2\xf5\xee\x00\x00\x00+\x02\x00\x00\x11\x00\x00\x00docProps/core.xml\xcd\x92\xc1J\xc40\x10\x86_Ero\'m`\x85\xd0\xedE\xf1\xa4 \xb8\xa0x\x0b\x93\xd9\xdd`\x93\x86d\xa4\xdd\xb7\xb7\xad\xbb]D\x1f\xc0cf\xfe|\xf3\rL\x83Qc\x9f\xe89\xf5\x91\x12;\xca7\xa3\xefB\xd6\x18\xb7\xe2\xc8\x1c5@\xc6#y\x93\xcb)\x11\xa6\xe6\xbeO\xde\xf0\xf4L\x07\x88\x06?\xcc\x81\xa0\x96r\x03\x9e\xd8X\xc3\x06f`\x11W\xa2h\x1b\x8b\x1a\x13\x19\xee\xd3\x19oq\xc5\xc7\xcf\xd4-0\x8

In [41]:
df = pd.read_excel('Dataset.xls')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 48 columns):
 #   Column                                                                                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                                                                                            --------------  ----- 
 0   Unnamed: 0                                                                                                                                                                        212 non-null    int64 
 1   Sector                                                                                                                                                                            212 non-null    object
 2   Last page                                                         

#Data Cleaning

After running df.info(), we noticed the following:

1. Column 0 is spurious and needs to be dropped
2. Last page is irrelevent and needs to be dropped

when it comes to columns with missing values in over 30% of the entries, we'll be dropping those columns as well. These columns include:
1. What is your current marital status? [Comment]
2. What is your main source of income? [Comment]
3. What type of income or financial support does your household receive? [Comment]
4. What is your current employment status? [Comment]



In [42]:
df = df.drop(columns=['Unnamed: 0','Last page','Have you smoked at least one full tobacco cigarette (excluding e-cigarettes) once or more in the past 30 days?','What is your current marital status? [Comment]', 'What is your main source of income? [Comment]','What type of income or financial support does your household receive? [Comment]','What is your current employment status? [Comment]'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 41 columns):
 #   Column                                                                                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                                                                                            --------------  ----- 
 0   Sector                                                                                                                                                                            212 non-null    object
 1   I see myself as someone who is extraverted, enthusiastic:                                                                                                                         212 non-null    object
 2   I see myself as someone who is critical, quarrelsome:             

Following the deletion of certain columns, we'll identify and drop 'redundant' columns with , if any.

In [43]:
# Select only numeric columns
numeric_df = df.select_dtypes(include=['number'])

# Calculate the correlation matrix
corr_matrix = numeric_df.corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find redundant columns
redundant_cols = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

# Drop redundant columns
df = df.drop(columns=redundant_cols)
print("Dropped columns:", redundant_cols)
print(df.info())

Dropped columns: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 41 columns):
 #   Column                                                                                                                                                                            Non-Null Count  Dtype 
---  ------                                                                                                                                                                            --------------  ----- 
 0   Sector                                                                                                                                                                            212 non-null    object
 1   I see myself as someone who is extraverted, enthusiastic:                                                                                                                         212 non-null    object
 2   I see myself as someone who is critical, quarr

Now we want to uniquely encode each non-numeric column with numerics.







In [44]:
def unique_encode_column(df, column_name):
    """
    Encodes a column in the dataframe with unique numeric values for each unique category.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    column_name (str): The column to be encoded.

    Returns:
    pd.DataFrame: The dataframe with the encoded column.
    dict: The mapping of original values to encoded values.
    """
    # Create a unique mapping for the column
    unique_values = df[column_name].unique()
    encoding_map = {value: idx for idx, value in enumerate(unique_values)}

    # Replace the column values with their encoded values
    df[column_name] = df[column_name].map(encoding_map)

    return df, encoding_map

columns_to_encode= list(df.columns)

all_encodings={}

for col in columns_to_encode:
    df, encoding_map = unique_encode_column(df, col)
    all_encodings[col] = encoding_map

# Example usage:

print(all_encodings)  # To see the updated dataframe



{'Sector': {'Private': 0, 'Public': 1}, 'I see myself as someone who is extraverted, enthusiastic:': {'Agree strongly': 0, 'Agree moderately': 1, 'Disagree moderately': 2, 'Agree a little': 3, 'Disagree strongly': 4, 'Disagree a little': 5, 'Neither agree nor disagree': 6}, 'I see myself as someone who is critical, quarrelsome:': {'Disagree moderately': 0, 'Agree moderately': 1, 'Neither agree nor disagree': 2, 'Agree a little': 3, 'Disagree a little': 4, 'Agree strongly': 5, nan: 6}, 'I see myself as someone who is dependable, self-disciplined:': {'Agree strongly': 0, 'Agree moderately': 1, 'Agree a little': 2, 'Neither agree nor disagree': 3, 'Disagree moderately': 4, 'Disagree a little': 5, 'Disagree strongly': 6}, 'I see myself as someone who is anxious, easily upset:': {'Agree moderately': 0, 'Agree a little': 1, 'Disagree strongly': 2, 'Disagree moderately': 3, 'Agree strongly': 4, 'Neither agree nor disagree': 5, 'Disagree a little': 6}, 'I see myself as someone who is open to n

Checking the final DataFrame

In [45]:
print(df.head())

   Sector  I see myself as someone who is extraverted, enthusiastic:  \
0       0                                                  0           
1       0                                                  1           
2       0                                                  2           
3       0                                                  0           
4       0                                                  3           

   I see myself as someone who is critical, quarrelsome:  \
0                                                  0       
1                                                  1       
2                                                  2       
3                                                  0       
4                                                  3       

   I see myself as someone who is dependable, self-disciplined:  \
0                                                  0              
1                                                  1              
2    